<a href="https://colab.research.google.com/github/agilan123/UIUC-Involvement-BOT/blob/main/MLModel1(Supra%20V%20-%200.6481481194496155).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from keras.layers import Bidirectional, LSTM, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten

df = pd.read_csv('Data.csv')
df.dropna(subset=['Intensity (1-3)'], inplace=True)

df

,Entries (What does the text say?),Classification,Intensity (1-3),Unnamed: 3
0,I participated in several business competition...,Business,3.0,NaN
1,"I really want to learn more about robotics, I ...",Robotics,3.0,NaN
2,I recently started learning about artifical in...,Machine Learning,1.0,NaN
3,I've been painting for about a year and want t...,Arts,2.0,NaN
4,I just heard about consulting and I would like...,Consulting,1.0,NaN
...,...,...,...,...
486,I'm just starting to learn about the blockchai...,Blockchain,1.0,NaN
487,I'm a bilangual student looking to learn more ...,Language,2.0,NaN
488,I'm a professional eSports member and I'm look...,Gaming,2.0,NaN
489,I'm looking to learn more about the medical field,Medicine,1.0,NaN


In [4]:

queries = df['Entries (What does the text say?)'].values
intensity = df['Intensity (1-3)'].values

X_train, X_test, y_train, y_test = train_test_split(queries, intensity, test_size=0.2, random_state=42)

from keras.utils import to_categorical

y_train = y_train - 1
y_test = y_test - 1

y_train_encoded = to_categorical(y_train, num_classes=3)
y_test_encoded = to_categorical(y_test, num_classes=3)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max(len(x) for x in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

priority_model = Sequential([
    Embedding(input_dim=100000, output_dim=16, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])


priority_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
priority_model.summary()

main_acc = 0
batch = 5

#for val in range(1,32):
#  history = priority_model.fit(X_train_padded, y_train, epochs=27, batch_size=val, validation_data=(X_test_padded, y_test))
#  test_loss, test_acc = priority_model.evaluate(X_test_padded, y_test, verbose=2)
#  if test_acc > main_acc:
#    main_acc = test_acc
#    batch = val

history = priority_model.fit(X_train_padded, y_train, epochs=27, batch_size = batch, validation_data=(X_test_padded, y_test))


test_loss, test_acc = priority_model.evaluate(X_test_padded, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


predictions = priority_model.predict(X_test_padded)
predicted_priorities = np.argmax(predictions, axis=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 16)            1600000   
                                                                 
 bidirectional (Bidirection  (None, 32, 128)           41472     
 al)                                                             
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score
import numpy as np

def create_model(optimizer='adam', hidden_layers=1, units=32, input_dim=100000, max_length=None):
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=16, input_length=max_length),
        GlobalAveragePooling1D(),
    ])

    for _ in range(hidden_layers):
        model.add(Dense(units, activation='relu'))

    model.add(Dense(5, activation='softmax'))  # Adjust for 5-class classification
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

keras_classifier = KerasClassifier(build_fn=lambda: create_model(input_dim=100000, max_length=max_length), epochs=10, batch_size=32, verbose=0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 3.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')